In [1]:
print("Start")
import torch
import os
import numpy as np
from transformers import (T5ForConditionalGeneration, 
AdamW, T5Tokenizer)
from peft import PeftConfig, PeftModel
from datasets import load_dataset, Dataset, DatasetDict
from torch.utils.data import DataLoader
import pandas as pd
print("Libraries Imported Successfully")


Start


/home/pravin/Desktop/T5/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries Imported Successfully


In [2]:
model_id = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model1 = PeftModel.from_pretrained(model, "/home/pravin/Desktop/April/7thapril/Sentiment/epoch1", map_location=torch.device('cpu'))
model2 = PeftModel.from_pretrained(model, "/home/pravin/Desktop/April/7thapril/Sentiment/second_iteration/epoch2", map_location=torch.device('cpu'))


In [4]:
device = next(model.parameters()).device
print("Model device:", device)

Model device: cpu


In [4]:
df = pd.read_csv("/home/pravin/Desktop/April/7thapril/Sentiment/test_amazon.csv")
df = df.sample(400)

In [5]:
def apply_model(row, modell):
    input_ids = tokenizer(f" question: {row['question']} context: {row['context']}", return_tensors='pt').input_ids
    output = tokenizer.decode(modell.generate(inputs = input_ids)[0], skip_special_tokens=True) 
    return output

In [6]:
df['m0'] = df.apply(lambda row: apply_model(row, model), axis=1)
df['m1'] = df.apply(lambda row: apply_model(row, model1), axis=1)
df['m2'] = df.apply(lambda row: apply_model(row, model2), axis=1)

/home/pravin/Desktop/T5/myenv/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/pravin/Desktop/T5/myenv/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/pravin/Desktop/T5/myenv/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
df.head()

,context,answer,question,m0,m1,m2
157084,Not worth it: They broke Day 1... ends came o...,negative,How does the customer feel?,negative,negative,negative
74635,TERRIBLE PRODUCT! COMPLETELY INEFFECTIVE!!!!!...,negative,Can you determine the sentiment expressed in t...,negative,negative,negative
342447,Not nearly good as the Broadway version: This...,negative,Do you think the text has a positive or negati...,negative,negative,negative
332791,Biased and poorly done: Minter is yet another...,negative,What is the emotional sentiment conveyed by th...,negative,negative,negative
295004,Tamboo/Skins: What a pleasure seeing these al...,positive,Sentiment?,positive,positive,positive


In [10]:
df['m0'].unique(), df['m1'].unique(), df['m2'].unique()

(array(['negative', 'positive', 'neutral'], dtype=object),
 array(['negative', 'positive', 'neutral'], dtype=object),
 array(['negative', 'positive', 'neutral'], dtype=object))

In [12]:
df['m088'] = df.apply(lambda row: apply_model(row, model), axis=1)


/home/pravin/Desktop/T5/myenv/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [13]:
df[df['m088'] == df['m0']].shape

(22, 7)

In [8]:
df_filtered0 = df[(df['answer'] != df['m0'])]

In [9]:
df_filtered0

,context,answer,question,m0,m1,m2
159791,teenybopper punk rock: It's hard to believe t...,positive,Sentiment?,negative,negative,negative
40558,Good but not as good as Trance to the Sun: I ...,negative,What sentiment does the text convey?,positive,positive,positive
29070,If you know absolutely nothing about ...: ......,negative,How does the customer feel?,positive,positive,positive
302933,Should have been the first recording.: Anothe...,positive,Can you determine the sentiment expressed in t...,negative,negative,negative
47960,Widescreen Letterbox Format: I fail to see th...,negative,What sentiment does the text convey?,positive,positive,positive
122569,The 50 states quarter folder: It could not se...,negative,Can you analyze the sentiment of the text?,neutral,neutral,neutral
105453,Pleasant collect for casual listening: A so s...,negative,Sentiment?,positive,positive,positive
106917,"Not what I expected.: Wonderful voice, no dou...",negative,What sentiment does the text convey?,positive,positive,positive
121736,oh man: i have not had the pleasure of listen...,negative,What sentiment does the text convey?,positive,positive,positive
20657,Why do they always have kool keith?: I don't ...,positive,Can you determine the sentiment expressed in t...,negative,negative,negative
